# Lastfm top interactive dashboard

## Ricardo Vargas (September 2023)

### Import libraries

In [1]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

### Load the CSV file

In [2]:
data = pd.read_csv("/Users/ricardo/Downloads/reduce_data.csv")
data.head()

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid
0,1694139763,"08 Sep 2023, 02:22",Sebadoh,d98c8149-0c01-4151-93e1-48fdc3d5027f,Bubble and Scrape,4cbb2d77-c4bd-4c4d-84c0-93c37246e755,Soul and Fire,330ff1a3-6284-3bf9-8815-3ae62904e5ec
1,1694045592,"07 Sep 2023, 00:13",boygenius,3ceeddbd-fba5-4bdb-99f7-2d028ed5afda,the record,b949ab39-bf54-4a99-aab0-dacc7d3e349a,Not Strong Enough,58389d98-bfc9-4956-9dc2-4a692525958b
2,1693842097,"04 Sep 2023, 15:41",R.E.M.,ea4dfa26-f633-4da6-a52a-f49ea4897b58,Lifes Rich Pageant,1249abaf-29b6-48e2-9095-b64f7b15e200,Hyena,a0f31742-c370-4fa6-a546-f88a91396bce
3,1693841613,"04 Sep 2023, 15:33",R.E.M.,ea4dfa26-f633-4da6-a52a-f49ea4897b58,Lifes Rich Pageant,1249abaf-29b6-48e2-9095-b64f7b15e200,Cuyahoga,024f5992-9309-3793-9e5e-a857d6b7cd13
4,1693838243,"04 Sep 2023, 14:37",R.E.M.,ea4dfa26-f633-4da6-a52a-f49ea4897b58,Lifes Rich Pageant,1249abaf-29b6-48e2-9095-b64f7b15e200,These Days,7602ce11-4797-41f6-afd3-dbd3c7820622


### Extract year and month

In [3]:
# Convert the 'utc_time' column to datetime format
data['utc_time'] = pd.to_datetime(data['utc_time'], format='%d %b %Y, %H:%M')

# Extract year and month from the 'utc_time' column
data['year'] = data['utc_time'].dt.year
data['month'] = data['utc_time'].dt.month
data.head()

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid,year,month
0,1694139763,2023-09-08 02:22:00,Sebadoh,d98c8149-0c01-4151-93e1-48fdc3d5027f,Bubble and Scrape,4cbb2d77-c4bd-4c4d-84c0-93c37246e755,Soul and Fire,330ff1a3-6284-3bf9-8815-3ae62904e5ec,2023,9
1,1694045592,2023-09-07 00:13:00,boygenius,3ceeddbd-fba5-4bdb-99f7-2d028ed5afda,the record,b949ab39-bf54-4a99-aab0-dacc7d3e349a,Not Strong Enough,58389d98-bfc9-4956-9dc2-4a692525958b,2023,9
2,1693842097,2023-09-04 15:41:00,R.E.M.,ea4dfa26-f633-4da6-a52a-f49ea4897b58,Lifes Rich Pageant,1249abaf-29b6-48e2-9095-b64f7b15e200,Hyena,a0f31742-c370-4fa6-a546-f88a91396bce,2023,9
3,1693841613,2023-09-04 15:33:00,R.E.M.,ea4dfa26-f633-4da6-a52a-f49ea4897b58,Lifes Rich Pageant,1249abaf-29b6-48e2-9095-b64f7b15e200,Cuyahoga,024f5992-9309-3793-9e5e-a857d6b7cd13,2023,9
4,1693838243,2023-09-04 14:37:00,R.E.M.,ea4dfa26-f633-4da6-a52a-f49ea4897b58,Lifes Rich Pageant,1249abaf-29b6-48e2-9095-b64f7b15e200,These Days,7602ce11-4797-41f6-afd3-dbd3c7820622,2023,9


### Make the selection widget

In [4]:
# Widgets for selecting year and month range
start_year = widgets.IntSlider(min=data['year'].min(), max=data['year'].max(), value=data['year'].min(), description='Start Year')
end_year = widgets.IntSlider(min=data['year'].min(), max=data['year'].max(), value=data['year'].max(), description='End Year')
start_month = widgets.IntSlider(min=1, max=12, value=1, description='Start Month')
end_month = widgets.IntSlider(min=1, max=12, value=12, description='End Month')

### Filtering data and graphs 

In [17]:
# Display function for the charts
def display_charts(start_y, end_y, start_m, end_m):
    # Filter data
    filtered_data = data[(data['year'] >= start_y) & (data['year'] <= end_y) & 
                         (data['month'] >= start_m) & (data['month'] <= end_m)]
    
    # Top Artists Chart
    top_artists = filtered_data['artist'].value_counts().head(20).reset_index()
    top_artists.columns = ['Artist', 'Count']
    fig1 = px.bar(top_artists, x='Artist', y='Count', title='Top 20 Most Listened Artists')
    fig1.show()
    
    # Top Albums Chart
    top_albums = filtered_data.groupby(['artist', 'album']).size().reset_index(name='Count')
    top_albums = top_albums.sort_values(by='Count', ascending=False).head(20)

    # Define the explicit order for the x-axis based on the album names in descending order of count
    album_order = top_albums['album'].tolist()

    fig2 = px.bar(top_albums, x='album', y='Count', color='artist',
                  title='Top 20 Most Listened Albums', labels={'album': 'Album', 'Count': 'Count'},
                  category_orders={"album": album_order})
    fig2.show()


    # Top Tracks Chart
    top_tracks = filtered_data.groupby(['artist', 'album', 'track']).size().reset_index(name='Count')
    top_tracks = top_tracks.sort_values(by='Count', ascending=False)
    top_tracks = top_tracks.head(20)

    # Define the explicit order for the x-axis based on the track names in descending order of count
    track_order = top_tracks['track'].tolist()

    fig3 = px.bar(top_tracks, x='track', y='Count', color='artist',
                  title='Top 20 Most Listened Tracks', labels={'track': 'Track', 'Count': 'Count'},
                  category_orders={"track": track_order})
    fig3.show()



    

### Interact widgets

In [18]:
widgets.interact(display_charts,
                 start_y=start_year,
                 end_y=end_year,
                 start_m=start_month,
                 end_m=end_month)

interactive(children=(IntSlider(value=2021, description='Start Year', max=2023, min=2007), IntSlider(value=202…

<function __main__.display_charts(start_y, end_y, start_m, end_m)>